# Differentiable Physics III: Navier Stokes Equations

In this notebook we look at the navier stokes equations and how to solve them numerically. The number of numerical methods for solving the navier stokes equations is large. 

We will look only at pressure correction method.

## Introduction
We ignore any external forces and assume that the fluid is incompressible.

Given:

- $\vec{v}$: velocity vector field with x / y components $u$ and $v$
- $p$: pressure vector field
- $\rho$: density
- $\nu$: kinematic viscosity

And:

- $\nabla$ is the gradient operator
- $\nabla^2$ or $\Delta$ is the Laplacian operator


The equations for the incompressible Navier Stokes equations are:

1. Momentum conservation:
$$\frac{\partial \mathbf{u}}{\partial t} + \vec{v} \cdot \nabla \vec{v} = -\frac{1}{\rho} \nabla p + \nu \Delta \vec{v}$$

2. Incompressibility or mass conservation:
$$\nabla \cdot \vec{v} = 0$$

We have the followin equivalence:

$$ \nabla \cdot \vec{v} = \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y}$$

And coresspondingly:

$$ \vec{v} \nabla \cdot \vec{v} = u \frac{\partial u}{\partial x} + v \frac{\partial v}{\partial y}$$


As we are interested in the 2D case, we can write the equations as:

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} = -\frac{1}{\rho} \frac{\partial p}{\partial x} + \nu \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right) \tag{1}$$

$$\frac{\partial v}{\partial t} + u \frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y} = -\frac{1}{\rho} \frac{\partial p}{\partial y} + \nu \left( \frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2} \right) \tag{2}$$

In the incompressible case the momentum conservation equation and the incompressibility equation are not coupled. We can couple them by taking the divergence of the momentum conservation equation and applying the incompressibility property:

$$\nabla \left(\frac{\partial \mathbf{u}}{\partial t} + \vec{v} \cdot \nabla \vec{v}\right) = \nabla \left( -\frac{1}{\rho} \nabla p + \nu \Delta \vec{v}  \right)$$

On the LHS the term $\nabla \cdot \vec{v}$ is zero, so we get:


$$\nabla \left(\vec{v} \cdot \nabla \vec{v}\right) = \nabla \left( -\frac{1}{\rho} \nabla p + \nu \Delta \vec{v} \right)$$

On the RHS $\nabla \nu \nabla^2 \vec{v}$ is zero, so we get:

$$\nabla \left(\vec{v} \cdot \nabla \vec{v}\right) = \nabla \left( -\frac{1}{\rho} \nabla p \right) = -\frac{1}{\rho} \Delta p$$

Which we can rearange to:

$$\Delta p = -\rho \nabla \left(\vec{v} \cdot \nabla \vec{v}\right)$$

Which can be written in 2D as:

$$\frac{\partial^2 p}{\partial x^2} + \frac{\partial^2 p}{\partial y^2} = -\rho \left(  \frac{\partial u}{\partial x} \frac{\partial u}{\partial x} + 2 \frac{\partial u}{\partial y} \frac{\partial v}{\partial x} + \frac{\partial v}{\partial y} \frac{\partial v}{\partial y} \right) \tag{3}$$


## Finite Difference Method and Discretization

We can approximate the derivatives of a function $f$ with respect to $x$ and $y$ as using the **forward difference** method:

$$\frac{\partial f}{\partial x} \approx \frac{f(x + \Delta x, y) - f(x, y)}{\Delta x}$$

When we use a uniform grid:

$$\frac{\partial f}{\partial x} \approx \frac{f_{i+1, j} - f_{i, j}}{\Delta x}$$

Analagously we can approximate the **backward difference** as:

$$\frac{\partial f}{\partial x} \approx \frac{f_{i, j} - f_{i-1, j}}{\Delta x}$$

And the **central difference** as:

$$\frac{\partial f}{\partial x} \approx \frac{f_{i+1, j} - f_{i-1, j}}{2 \Delta x}$$

We can compose these approximations to get the following approximations for the second derivatives. The **second order central difference**:

$$\frac{\partial^2 f}{\partial x^2} \approx \frac{f_{i+1, j} - 2 f_{i, j} + f_{i-1, j}}{\Delta x^2}$$


Solvers for the Navier Stokes equations are usually based on the **projection method**. The projection method is based on the following steps:

1. Solve the momentum equation for the intermediate velocity field $\vec{v}^*$ using a forward time step:

$$\frac{\vec{v}^* - \vec{v}^n}{\Delta t} = -\frac{1}{\rho} \nabla p + \nu \Delta \vec{v}^n$$

2. Enforce the incompressibility condition by projecting the intermediate velocity field onto the space of divergence-free vector fields:

$$\vec{v}^{n+1} = \vec{v}^* - \nabla \phi$$

3. Solve the Poisson equation for the pressure field $\phi$:

$$\Delta \phi = \frac{\nabla \cdot \vec{v}^*}{\Delta t}$$

4. Update the pressure field:

$$p^{n+1} = p^n + \phi$$


### Discretization of the Navier Stokes equations

## Lax Wendroff Method

Generally we make use of the finite difference method to discretize the equations. For the partial derivatives we use the central difference method. For the time derivative we use the forward difference method.

Therefore the gradient of a scalar field $f$ is:

$$\nabla f = \left( \frac{\partial f}{\partial x}, \frac{\partial f}{\partial y} \right) \approx \left( \frac{f_{i+1,j} - f_{i-1,j}}{2 \Delta x}, \frac{f_{i,j+1} - f_{i,j-1}}{2 \Delta y} \right)$$


Lets discretize the u momentum conservation equation (1) (The v momentum conservation equation (2) is analogous):

$$\frac{u^{n+1}_{i,j} - u^n_{i,j}}{\Delta t} + u^n_{i,j} \frac{u^n_{i,j} - u^n_{i-1,j}}{\Delta x} + v^n_{i,j} \frac{u^n_{i,j} - u^n_{i,j-1}}{\Delta y} = \\
-\frac{1}{\rho} \frac{p^{n+1}_{i+1,j} - p^{n+1}_{i-1,j}}{ 2\Delta x} + \nu \left( \frac{u^n_{i+1,j} - 2 u^n_{i,j} + u^n_{i-1,j}}{\Delta x^2} + \frac{u^n_{i,j+1} - 2 u^n_{i,j} + u^n_{i,j-1}}{\Delta y^2} \right)$$

We can reformulate this equation to get an expression for $u^{n+1}_{i,j}$ (the only unknown in the equation):

$$u^{n+1}_{i,j} = u^n_{i,j} - u^n_{i,j} \frac{\Delta t \left( u^n_{i,j} - u^n_{i-1,j} \right) }{\Delta x} - v^n_{i,j} \frac{\Delta t \left( u^n_{i,j} - u^n_{i,j-1} \right) }{\Delta y} - \frac{\Delta t \left( p^{n+1}_{i+1,j} - p^{n+1}_{i-1,j} \right)}{\rho 2 \Delta x} + \frac{\nu \Delta t \left( u^n_{i+1,j} - 2 u^n_{i,j} + u^n_{i-1,j} \right)}{\Delta x^2}  + \frac{\nu \Delta t \left( u^n_{i,j+1} - 2 u^n_{i,j} + u^n_{i,j-1} \right)}{\Delta y^2} $$



We can discretize the pressure Poisson equation (3) as follows:

$$\frac{p^{n+1}_{i+1,j} - 2 p^{n+1}_{i,j} + p^{n+1}_{i-1,j}}{\Delta x^2} + \frac{p^{n+1}_{i,j+1} - 2 p^{n+1}_{i,j} + p^{n+1}_{i,j-1}}{\Delta y^2} = \\

-\rho \biggl(  \frac{1}{\Delta t} \Bigl(\frac{u^n_{i+1,j} - u^n_{i-1,j}}{2 \Delta x} + \frac{u^n_{i,j+1} - u^n_{i,j-1}}{2 \Delta y} \Bigr)
- \Bigl(\frac{u^n_{i+1,j} - u^n_{i-1,j}}{\Delta x}\Bigr)^2
- 2 \frac{u^n_{i,j+1} - u^n_{i,j-1}}{\Delta y} \frac{u^n_{i+1,j} - u^n_{i-1,j}}{\Delta x}
- \Bigl(\frac{u^n_{i,j+1} - u^n_{i,j-1}}{\Delta y}\Bigr)^2 \biggr)$$


Which we can reformulate to get an expression for $p^{n}_{i,j}$ (the only unknown in the equation):

$$p^{n}_{i,j} = 
- \frac{\rho \Delta x^2 \Delta y^2}{2 \left( \Delta x^2 + \Delta y^2 \right)} \Biggl( \frac{1}{\Delta t}\Bigl(\frac{u^n_{i+1,j} - u^n_{i-1,j}}{2 \Delta x} + \frac{u^n_{i,j+1} - u^n_{i,j-1}}{2 \Delta y} \Bigr)
- \Bigl(\frac{u^n_{i+1,j} - u^n_{i-1,j}}{\Delta x}\Bigr)^2
- 2 \frac{u^n_{i,j+1} - u^n_{i,j-1}}{\Delta y} \frac{u^n_{i+1,j} - u^n_{i-1,j}}{\Delta x}
- \Bigl(\frac{u^n_{i,j+1} - u^n_{i,j-1}}{\Delta y}\Bigr)^2 \Biggr) + \\ 
\frac{\bigl(p^{n}_{i+1,j} + p^{n}_{i-1,j}\bigr) \Delta y^2 + \bigl(p^{n}_{i,j+1} + p^{n}_{i,j-1  }\bigr) \Delta x^2}{2 \bigl(\Delta x^2 + \Delta y^2 \bigr)}$$




## MacCormack Method

The MacCormack method is a predictor-corrector method. It is a two step method. In the first step we predict the values of the variables at the next time step. In the second step we correct the predicted values.



## Solving the equations

We can solve the equations by iterating over the equations and solving them for the unknowns. We can use the Jacobi method to solve the equations. The Jacobi method is an iterative method that solves a system of linear equations by repeatedly replacing each diagonal element by the sum of the non-diagonal elements and the right hand side divided by the diagonal element. This method is guaranteed to converge if the matrix is diagonally dominant. The matrix of the discretized equations is diagonally dominant if the viscosity is sufficiently large. The Jacobi method is not the most efficient method to solve the equations, but it is easy to implement.

In [8]:
import jax.numpy as jnp
from functools import partial
from jax import jit

@partial(jit, static_argnames='reflect')
def get_surround(x, reflect : bool = True) -> tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray, jnp.ndarray]:
    x_l = jnp.roll(x, -1, axis=0)
    x_r = jnp.roll(x, 1, axis=0)
    x_b = jnp.roll(x, -1, axis=1)
    x_t = jnp.roll(x, 1, axis=1)

    if not reflect:
        x_l = x_l.at[-1, :].set(0)
        x_r = x_r.at[0, :].set(0)
        x_b = x_b.at[:, -1].set(0)
        x_t = x_t.at[:, 0].set(0)

    return x_l, x_r, x_b, x_t

In [9]:
@partial(jit, static_argnames=('dt', 'dx', 'dy'))
def pressure_step(u : jnp.ndarray, p : jnp.ndarray, dx : float, dy : float, dt : float) -> jnp.ndarray:
    u_l, u_r, u_b, u_t = get_surround(u)
    p_l, p_r, p_b, p_t = get_surround(p)

    a = 1 / dt * ((u_r - u_l) / (2 * dx) + (u_t - u_b) / (2 * dy))
    b = 1 / dx**2 * ((u_r - u_l) / dx) ** 2
    c = 1 / dy**2 * ((u_t - u_b) / dy) ** 2
    d = 2 * ((u_r - u_l) / dx  * (u_t - u_b) / dy)
    e = p  * dx ** 2 * dy ** 2 / ( 2 * (dx ** 2 + dy ** 2))
    f = ((p_r + p_l) * dy ** 2  + (p_t + p_b) * dx ** 2) / (2 * (dx ** 2 + dy ** 2))

    return (a - b - c - d) * e + f

In [10]:
@partial(jit, static_argnames=('dt', 'dx', 'dy'))
def momentum_step(u : jnp.ndarray, p : jnp.ndarray, dx : float, dy : float, dt : float) -> jnp.ndarray:
    u_l, u_r, u_b, u_t = get_surround(u)
    p_l, p_r, p_b, p_t = get_surround(p)

    
    pass

In [11]:
@partial(jit, static_argnames=('dt', 'dx', 'dy', 'pressure_steps'))
def solver_step(u : jnp.ndarray, p : jnp.ndarray, dx : float, dy : float, dt : float, pressure_steps : int) -> tuple[jnp.ndarray, jnp.ndarray]:
    u = momentum_step(u, p, dx, dy, dt)

    for _ in range(pressure_steps):
        p = pressure_step(u, p, dx, dy, dt)
        
    return u, p

Sources: 

- https://digitalcommons.georgiasouthern.edu/cgi/viewcontent.cgi?article=1846
- https://nbviewer.org/github/barbagroup/CFDPython/blob/master/lessons/12_Step_9.ipynb
